# **Import Some Packages**

In [2]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For data preprocess
import pandas as pd
import numpy as np
import csv
import os

from tqdm import tqdm

import math
from torch.utils.tensorboard import SummaryWriter
# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

myseed = 80215  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# **Some Utilities**

You do not need to modify this part.

In [3]:
def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def plot_learning_curve(loss_record, title=''):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    x_2 = x_1[::len(loss_record['train']) // len(loss_record['dev'])]
    figure(figsize=(6, 4))
    plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
    plt.plot(x_2, loss_record['dev'], c='tab:cyan', label='dev')
    plt.ylim(1000.0, 100000.)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()


def plot_pred(dv_set, model, device, lim=35., preds=None, targets=None):
    ''' Plot prediction of your DNN '''
    if preds is None or targets is None:
        model.eval()
        preds, targets = [], []
        for x, y in dv_set:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                preds.append(pred.detach().cpu())
                targets.append(y.detach().cpu())
        preds = torch.cat(preds, dim=0).numpy()
        targets = torch.cat(targets, dim=0).numpy()

    figure(figsize=(5, 5))
    plt.scatter(targets, preds, c='r', alpha=0.5)
    plt.plot([-0.2, lim], [-0.2, lim], c='b')
    plt.xlim(-0.2, lim)
    plt.ylim(-0.2, lim)
    plt.xlabel('ground truth value')
    plt.ylabel('predicted value')
    plt.title('Ground Truth v.s. Prediction')
    plt.show()
    


In [4]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

In [5]:
attri_data = pd.read_csv('../LiterallyWikidata/files_needed/numeric_literals_ver06')

In [6]:
attri_data

,e,a,v,name_e,name_a,ent_type,std_v,minmax_v
0,Q1000056,P1082,1.103200e+04,Sušice,population,Q7841907,-0.030708,2.649215e-06
1,Q1000056,P2044,4.720000e+02,Sušice,elevation above sea level,Q7841907,0.031306,2.219876e-03
2,Q1000056,P2046,4.563000e+07,Sušice,area,Q7841907,-0.006852,7.492611e-12
3,Q1000138,P1082,1.375000e+03,Cantenac,population,Q484170,-0.031014,3.301914e-07
4,Q1000138,P2044,1.000000e+00,Cantenac,elevation above sea level,Q484170,-0.116549,1.058141e-03
...,...,...,...,...,...,...,...,...
296303,Q99987,P1333_Longtiude,9.586108e+00,Brembate di Sopra,coordinates of southernmost point,Q747074,-0.140675,5.334437e-01
296304,Q99987,P1334_Longtiude,9.595788e+00,Brembate di Sopra,coordinates of easternmost point,Q747074,-0.144653,5.264729e-01
296305,Q99987,P1335_Longtiude,9.564391e+00,Brembate di Sopra,coordinates of westernmost point,Q747074,-0.142728,5.252904e-01
296306,Q99987,P625_Longtiude,9.580647e+00,Brembate di Sopra,coordinate location(logtitude),Q747074,0.130122,5.244460e-01


In [7]:
# 用kgeemb順序
ent2idx ={}
with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as fr:
    for i, word in enumerate(fr.readlines()):
        ent2idx[word.strip()] = i


attri_data_std_v = attri_data[['e','a','std_v']]
attri_data_eav = attri_data[['e','a','v']]
# att2idx = {}
# #rel2idx = {v:k for k,v in enumerate(relations['label'].unique())}

# with open('../LiterallyWikidata/files_needed/attribute.txt','r') as fr:
#     for i, word in enumerate(fr.readlines()):
#         att2idx[word.strip()] = i
        
att2idx = {v:k for k,v in enumerate(attri_data['a'].unique())}

In [8]:
# attri_data['a_idx']=attri_data['a'].map(att2idx)
# attri_data['e_idx']=attri_data['e'].map(ent2idx)

In [9]:
attri_data_eav

,e,a,v
0,Q1000056,P1082,1.103200e+04
1,Q1000056,P2044,4.720000e+02
2,Q1000056,P2046,4.563000e+07
3,Q1000138,P1082,1.375000e+03
4,Q1000138,P2044,1.000000e+00
...,...,...,...
296303,Q99987,P1333_Longtiude,9.586108e+00
296304,Q99987,P1334_Longtiude,9.595788e+00
296305,Q99987,P1335_Longtiude,9.564391e+00
296306,Q99987,P625_Longtiude,9.580647e+00


In [10]:
def numeric_literal_array(data, ent2idx, att2idx):
    #'LiterallyWikidata/LitWD48K/train_attri_data'
    df_all = data

    # Resulting file
    num_lit = np.zeros([len(ent2idx), len(att2idx)], dtype=np.float32)

# Create literal wrt vocab
    for i, (s, p, lit) in enumerate(df_all.values):
        try:
            num_lit[ent2idx[s], att2idx[p]] = lit
        except KeyError:
            continue
    return num_lit


# num_lit shape (47998, 86)


In [11]:
num_lit = numeric_literal_array(attri_data_eav, ent2idx, att2idx)
print(num_lit.shape)

(47998, 86)


In [18]:
num_lit_stdv = numeric_literal_array(attri_data_std_v, ent2idx, att2idx)
print(num_lit_stdv.shape)

(47998, 86)


In [19]:
# ## constraint needed:
pop_idx = att2idx['P1082']
gdp = att2idx['P4010']
nominal_gdp = att2idx['P2131']
# nominal_gdp_per = att2idx['P2132']
gdp_per = att2idx['P2299']
# date_of_birth = att2idx['P569']
# date_of_death = att2idx['P570']
# area = ['P2046']
# # net_profit = att2idx['P2295']
# # retirement_age = att2idx['P3001']
# # age_of_majority = att2idx['P2997']
# # work_start = att2idx['P2031']
# # work_end = att2idx['P2032']

In [13]:
ent2idx['Q1000']

84

In [14]:
attri_data[attri_data['a']=='P1082']

,e,a,v,name_e,name_a,ent_type,std_v,minmax_v
0,Q1000056,P1082,11032.0,Sušice,population,Q7841907,-0.030708,2.649215e-06
3,Q1000138,P1082,1375.0,Cantenac,population,Q484170,-0.031014,3.301914e-07
6,Q100013,P1082,4109.0,Brembilla,population,Q1134686,-0.030927,9.867318e-07
9,Q100015,P1082,6009.0,Brignano Gera d'Adda,population,Q747074,-0.030867,1.442996e-06
12,Q100016,P1082,119.0,Brumano,population,Q747074,-0.031053,2.857656e-08
...,...,...,...,...,...,...,...,...
123807,Q99983,P1082,706.0,Bracca,population,Q747074,-0.031035,1.695383e-07
123810,Q99985,P1082,712.0,Branzi,population,Q747074,-0.031035,1.709791e-07
123813,Q99986,P1082,8551.0,Brembate,population,Q747074,-0.030787,2.053430e-06
123816,Q99987,P1082,7868.0,Brembate di Sopra,population,Q747074,-0.030808,1.889415e-06


In [20]:
num_lit[84][gdp_per]*num_lit[84][pop_idx]
num_lit[84][gdp]

36681910000.0

In [72]:
# x_list: ent的gdp有值，把除了gdp那個值之外的值存到inner_x_list，len(x_list)是有幾組變數有值
x_list=[]

for ent in num_lit_stdv:
    if ent[gdp] == 0:
        pass
    else:
        #print(ent[gdp])
        inner_x_list=[]
        for j in range(len(ent)):
            if j != gdp :
                inner_x_list.append(ent[j])
        inner_x_list.append(ent[gdp])
        x_list.append(inner_x_list)


In [73]:
x_list[0]

[10.250006,
 0.0,
 -0.006613904,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.5218797,
 10.139478,
 2.0880435,
 -0.21222888,
 0.96752954,
 1.7440671,
 -1.8439026,
 -0.7418871,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.3853737,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.10251015,
 4.6604037,
 -2.7036817,
 0.35937592,
 3.820149,
 -1.6484872,
 -10.671047,
 -7.5986323,
 -5.1867476,
 -12.194551,
 8.661657]

In [22]:
y_list=list()
for ent2 in num_lit:
    if ent2[gdp] ==0:
        pass
    else:
        y_list.append(ent2[gdp])

In [23]:
for i in range(len(x_list)):
    inner_x = x_list[i]
    inner_x.append(y_list[i])
x_list.append(inner_x)


In [54]:
select_feature = []
for i in range(len(x_list[0])-1):
    if x_list[0][i] !=0:
        select_feature.append(i)

In [55]:
select_feature

[0,
 2,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 65,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84]

In [33]:
attri_data[attri_data['a']=='P4010']

,e,a,v,name_e,name_a,ent_type,std_v,minmax_v
156,Q1000,P4010,3.668191e+10,Gabon,GDP (PPP),Q3624078,-0.225985,0.001880
414,Q1005,P4010,3.569106e+09,The Gambia,GDP (PPP),Q6256,-0.241117,0.000181
461,Q1006,P4010,2.857575e+10,Guinea,GDP (PPP),Q6256,-0.229690,0.001464
519,Q1007,P4010,3.171312e+09,Guinea-Bissau,GDP (PPP),Q6256,-0.241299,0.000160
588,Q1008,P4010,9.583674e+10,Ivory Coast,GDP (PPP),Q7270,-0.198953,0.004916
...,...,...,...,...,...,...,...,...
121853,Q971,P4010,2.869352e+10,Republic of the Congo,GDP (PPP),Q6256,-0.229636,0.001470
122011,Q974,P4010,7.231902e+10,Democratic Republic of the Congo,GDP (PPP),Q6256,-0.209700,0.003709
122243,Q977,P4010,2.342711e+09,Djibouti,GDP (PPP),Q6256,-0.241678,0.000118
122774,Q983,P4010,3.098132e+10,Equatorial Guinea,GDP (PPP),Q3624078,-0.228591,0.001588


In [42]:
def select_feat(train_data, valid_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid = train_data[:,:-1], valid_data[:,:-1]

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = select_feature # TODO: Select suitable feature columns.
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], y_train, y_valid

In [43]:
class KGMTL_Data(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
        

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)


    

In [74]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
# train_data, test_data = pd.read_csv('../LiterallyWikidata/files_needed/train_attri_data.csv',sep='\t').values, 
# pd.read_csv('../LiterallyWikidata/files_needed/test_attri_data.csv',sep='\t').values
train_data, valid_data = train_valid_split(x_list, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} """)
# test_data size: {test_data.shape}""")


# Select features
x_train, x_valid, y_train, y_valid = select_feat(train_data, valid_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset = KGMTL_Data(x_train, y_train), \
                                            KGMTL_Data(x_valid, y_valid)

print('train_dataset', train_dataset[0])

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
# test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (141, 86) 
valid_data size: (15, 86) 
number of features: 21
train_dataset (tensor([ 0.1144, -0.0069,  0.6703, -0.1330,  0.6834,  0.0256,  0.4453,  2.2679,
         0.6065, -0.5695,  0.5956, -0.8998, -2.0334, -2.2693, -2.2668, -2.2481,
         0.7553,  2.2824,  2.5167,  2.4367,  2.3661]), tensor(-0.1065))


In [57]:
train_dataset[5]

(tensor([ 1.5015e+00, -6.8378e-03, -1.1856e+00, -1.5706e-01, -5.6475e-01,
          9.5041e-01, -5.5587e-01, -6.9429e-01,  6.0650e-01,  1.0090e+00,
          4.6787e-01, -3.0733e+00, -6.0224e+00, -7.6364e+00, -6.3457e+00,
         -7.0670e+00,  5.9746e-01,  1.5089e+00,  1.9251e+00,  1.9837e+00,
          1.5109e+00]),
 tensor(1.6363e+11))

In [75]:
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(input_dim, 21),
            nn.Linear(21, 1)
        )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        x = self.layers(x)
        x = x.squeeze(1)
        return x

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        # TODO: you may implement L1/L2 regularization here
        return self.criterion(pred, target)

# **Preprocess**

We have three kinds of datasets:
* `train`: for training
* `dev`: for validation
* `test`: for testing (w/o target value)

In [19]:
num_lit[:,0]

array([0.0000000e+00, 6.2448000e+04, 3.2514595e+08, ..., 0.0000000e+00,
       9.0200000e+02, 2.5440000e+03], dtype=float32)

In [89]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9, weight_decay=1e-6) 

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    writer = SummaryWriter() # Writer of tensoboard
    if not os.path.isdir('./models_var'):
        os.mkdir('./models_var') # Create directory of saving models.
    
    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)   
            print(f'-------predict: {pred}, y: {y}----------') 
            #x_constraint = torch.tensor([ (y[i] - x[i][0]*x[i][18]) ** 2 for i in range(len(x))])
            x_constraint = torch.tensor([x[i][pop_idx]*x[i][gdp_per] for i in range(len(x))])
            x_constraint = x_constraint.to(device)          
            loss = criterion(pred, y) + criterion(pred, x_constraint)
            #loss = criterion(pred, y) 
            # criterion(pred,x_constraint)
                # ((pred-x[0]*x[18])**2) 

            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                # print(f'x: {x}')
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## **Validation**

In [26]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)# forward pass (compute output)
            #print(pred)
            mse_loss = model.cal_loss(pred, y)  # compute loss
            print(mse_loss)
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss

## **Testing**

In [25]:
def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []
    for x in tt_set:                            # iterate through the dataloader
        x = x.to(device)                        # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())   # collect prediction
    preds = torch.cat(preds, dim=0).numpy()     # concatenate all predictions and convert to a numpy array
    return preds

In [79]:
device = get_device()                 # get the current available device ('cpu' or 'cuda')
#os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'seed': 802,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'n_epochs': 50,                # maximum number of epochs
    'batch_size': 16,               # mini-batch size for dataloader
    'learning_rate':1e-3,
    'early_stop': 200,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': './models_var/model_gdp.pt' , # your model will be saved here
    'valid_ratio': 0.1,   # validation_size = train_size * valid_ratio
}


# **Load data and model**

In [77]:
model = NeuralNet(input_dim=x_train.shape[1]).to(device)  # Construct model and move to device
print(model)

NeuralNet(
  (layers): Sequential(
    (0): Linear(in_features=21, out_features=21, bias=True)
    (1): Linear(in_features=21, out_features=1, bias=True)
  )
  (criterion): MSELoss()
)


# **Start Training!**

In [80]:
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/50]: 100%|███████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 91.20it/s, loss=0.0585]


-------predict: tensor([-0.2327, -0.1981, -0.2078, -0.1650, -0.1732,  1.2533, -1.4899, -0.0776,
        -0.1578,  0.0718,  0.1080, -0.1879,  0.0512, -0.2842, -0.2741, -0.2617],
       device='cuda:0', grad_fn=<SqueezeBackward1>), y: tensor([-0.2297, -0.2217, -0.1065, -0.2112, -0.2021,  6.7906, -0.2388, -0.2264,
        -0.2400, -0.2229,  0.0460, -0.2375,  0.5884, -0.2423, -0.2424, -0.2417],
       device='cuda:0')----------
-------predict: tensor([ 0.1508,  0.3071,  0.0611,  0.1954, -0.1679, -0.2781, -0.9161, -0.1068,
        -2.2827,  0.1972,  2.5341, -0.3767,  0.5809, -0.1598, -0.1801, -0.4350],
       device='cuda:0', grad_fn=<SqueezeBackward1>), y: tensor([-1.5173e-01,  5.7982e-03, -1.6797e-01, -2.3327e-01, -1.9977e-01,
        -2.3402e-01, -2.4260e-01, -2.3112e-01, -2.4263e-01, -9.2797e-02,
         8.6617e+00,  2.7475e-01, -1.5453e-01,  1.4033e-02, -2.4112e-01,
        -1.0634e-01], device='cuda:0')----------
-------predict: tensor([ 0.0620,  0.7989, -0.0406, -0.1372, -0.0136, -0

Epoch [2/50]: 100%|████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 104.92it/s, loss=1.15]


-------predict: tensor([-0.1476,  0.1343,  0.1591, -0.0868, -0.1948, -0.3595, -0.0384, -0.1426,
        -0.2701,  0.0313, -0.2345, -0.0672, -0.2365, -0.4181, -0.1753, -0.2417],
       device='cuda:0', grad_fn=<SqueezeBackward1>), y: tensor([-0.1998, -0.2308, -0.2336, -0.2327, -0.2260, -0.1139, -0.1968, -0.2246,
        -0.2066, -0.2416, -0.2337, -0.2326, -0.2246, -0.2398, -0.2177, -0.2335],
       device='cuda:0')----------
-------predict: tensor([ 0.0194, -0.1646,  0.1213, -0.3015, -0.2862, -0.3274, -0.0319, -0.1301,
         0.0492,  1.8653, -0.4516, -0.6239,  0.0713, -0.4772, -0.2487, -0.0493],
       device='cuda:0', grad_fn=<SqueezeBackward1>), y: tensor([-0.2348, -0.2261, -0.2412, -0.2112,  0.2747, -0.2244, -0.0922, -0.2045,
        -0.0344,  6.7906, -0.2425, -0.2427, -0.2134, -0.2373, -0.2340, -0.2258],
       device='cuda:0')----------
-------predict: tensor([-0.1093, -0.0618,  0.0099, -0.2971,  0.9524, -0.2090,  3.6092, -0.2610,
        -0.1872, -0.5036, -0.1397, -0.6367,  0.1

Epoch [3/50]:   0%|                                                                                | 0/9 [00:00<?, ?it/s]

...

-------predict: tensor([-0.2116, -0.1632, -0.3762, -0.1820,  0.2592, -0.2862, -0.1674, -0.1980,
        -0.2317, -0.3042, -0.2748, -0.0208, -0.1840, -0.0998,  0.2498, -0.2694],
       device='cuda:0', grad_fn=<SqueezeBackward1>), y: tensor([-0.2240, -0.2415, -0.2336, -0.1932,  0.1581, -0.2391, -0.1968, -0.2245,
        -0.2258, -0.2410, -0.2393,  0.0753, -0.2045, -0.1409,  0.2747, -0.2335],
       device='cuda:0')----------


Epoch [50/50]:   0%|                                                                 | 0/9 [00:00<?, ?it/s, loss=0.00296]

-------predict: tensor([-0.0668, -0.2313, -0.1741, -0.1885, -0.2627, -0.0629, -0.1384, -0.1740,
        -0.2650,  0.0388, -0.2827, -0.0907, -0.0760], device='cuda:0',
       grad_fn=<SqueezeBackward1>), y: tensor([-0.1678, -0.2325, -0.1750, -0.1642, -0.2217, -0.1063, -0.2273, -0.2249,
        -0.2313,  0.0535, -0.2044, -0.1680, -0.0922], device='cuda:0')----------


Epoch [50/50]: 100%|█████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 96.71it/s, loss=0.00296]

Epoch [50/50]: Train loss: 0.0109, Valid loss: 0.0176


In [87]:
!kill 1489040

In [88]:
#%reload_ext tensorboard
%tensorboard --logdir=./runs/

In [ ]:
plot_learning_curve(model_loss_record, title='deep model')

In [21]:
model_loss

1e+20

In [ ]:
del model
model = NeuralNet(tr_set.dataset.dim).to(device)
ckpt = torch.load(config['save_path'], map_location='cpu')  # Load your best model
model.load_state_dict(ckpt)
plot_pred(dv_set, model, device)  # Show prediction on the validation set

# **Testing**
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

preds = test(valid_loader, model, device)  # predict COVID-19 cases with your model
print('pred')         # save prediction file to pred.csv

# **Hints**

## **Simple Baseline**
* Run sample code

## **Medium Baseline**
* Feature selection: 40 states + 2 `tested_positive` (`TODO` in dataset)

## **Strong Baseline**
* Feature selection (what other features are useful?)
* DNN architecture (layers? dimension? activation function?)
* Training (mini-batch? optimizer? learning rate?)
* L2 regularization
* There are some mistakes in the sample code, can you find them?

# **Reference**
This code is completely written by Heng-Jui Chang @ NTUEE.  
Copying or reusing this code is required to specify the original author. 

E.g.  
Source: Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)


In [94]:
## Load pretrain embedding
emb_ent = torch.load('../LiterallyWikidata/files_needed/pretrained_kge/pretrained_complex_entemb.pt')
list_ent_ids =[]
with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as f:
    for line in f:
        list_ent_ids.append(line.strip())
## Preparing ent embedding
ent2idx = {e:i for i,e in enumerate(list_ent_ids)}
attri_data['ent_idx']= attri_data['e'].map(ent2idx)
embedding_e = torch.nn.Embedding.from_pretrained(emb_ent)
input_e = torch.LongTensor(attri_data['ent_idx'].to_numpy())

entity_embedding = embedding_e(input_e)
## Preparing att embedding
# att2idx = {a:i for i,a in enumerate(attri_data['a'].unique())}
attri_data['a_idx']=attri_data['a'].map(att2idx)
embedding_a = torch.nn.Embedding(len(att2idx),128,padding_idx=0)
input_a = torch.LongTensor(attri_data['a'].to_numpy())

attribute_embedding = embedding_a(input_a)
## concat two embedding
x_data = torch.cat([entity_embedding,attribute_embedding],dim=1).detach().numpy()

